In [1]:
from pathlib import Path
import duckdb
import pandas as pd

data_root = Path("D:\\데이터\\건축데이터 건축허브 개방데이터")
path_DB = data_root / "건축물대장_2025년_02월.db"

건축허브에서 제공하는 건축물대장 데이터를 활용하여 분석

2025년 2월 이전 데이터는 위반건축물 등 일부 건축물이 빠져있는 문제가 있어 부득이 2025년 2월 말 데이터로 2024년 말 기준 건축물 현황을 집계

In [2]:
# Open a DuckDB connection
con = duckdb.connect(database=path_DB, read_only=True)

# print the list of tables in the database
tables = con.execute("SHOW TABLES").fetchall()
print("Tables in the database:")
for table in tables:
    print(table[0])
    print()

# show heads of the tables
for table in tables:
    table_name = table[0]
    print(f"Head of {table_name}:")
    df = con.execute(f"SELECT * FROM {table_name} LIMIT 5").fetchdf()
    print(df)
    print()

Tables in the database:
기본개요

층별개요

표제부

Head of 기본개요:
  관리_건축물대장_PK 관리_상위_건축물대장_PK 대장_구분_코드 대장_구분_코드_명 대장_종류_코드 대장_종류_코드_명  \
0      100211           None        1         일반        1      총괄표제부   
1      100215           None        1         일반        1      총괄표제부   
2      100216           None        1         일반        1      총괄표제부   
3      100218           None        1         일반        1      총괄표제부   
4      100219           None        1         일반        1      총괄표제부   

                  대지_위치                  도로명_대지_위치  건물_명 시군구_코드  ...  \
0  서울특별시 종로구 관철동 12-1번지   서울특별시 종로구 종로14길 20 (관철동)  None  11110  ...   
1  서울특별시 종로구 청진동 21-1번지  서울특별시 종로구 종로5길 32-6 (청진동)  None  11110  ...   
2  서울특별시 종로구 중학동 16-2번지                       None  None  11110  ...   
3  서울특별시 종로구 중학동 24-1번지      서울특별시 종로구 율곡로 5 (중학동)  None  11110  ...   
4  서울특별시 종로구 중학동 35-1번지                       None  None  11110  ...   

  새주소_지상지하_코드 새주소_본_번 새주소_부_번 지역_코드 지구_코드 구역_코드 지역_코드_명  지구_코드_명 구역_코드_명  \
0  

In [3]:
# Create a list to store the table names and their record counts
table_counts = []

# Iterate through the tables and count the records
for table in tables:
    table_name = table[0]
    count = con.execute(f"SELECT COUNT(*) FROM {table_name}").fetchone()[0]
    table_counts.append({"Table Name": table_name, "Record Count": count})

# Convert the list to a DataFrame
record_counts_df = pd.DataFrame(table_counts)

# Display the DataFrame
display(record_counts_df)

,Table Name,Record Count
0,기본개요,27994157
1,층별개요,21048715
2,표제부,8027067


각 테이블별 레코드 수 확인

In [4]:
# # Create a dictionary to store the results
# max_lengths = {}

# # Iterate through the tables
# for table in tables:
#     table_name = table[0]
#     # Fetch the table schema
#     schema = con.execute(f"DESCRIBE {table_name}").fetchdf()
#     # display(schema)
#     # Filter text columns
#     text_columns = schema[
#         schema["column_type"].str.contains("VARCHAR|TEXT", case=False)
#     ]["column_name"]
#     # Initialize a dictionary for the current table
#     max_lengths[table_name] = {}
#     # Calculate max length for each text column
#     for column in text_columns:
#         max_length = con.execute(
#             f"SELECT MAX(LENGTH({column})) FROM {table_name}"
#         ).fetchone()[0]
#         max_lengths[table_name][column] = max_length

# # Display the results
# for table_name, columns in max_lengths.items():
#     print(f"Table: {table_name}")
#     for column, length in columns.items():
#         print(f"  Column: {column}, Max Length: {length}")
#     print()

각 테이블별, 컬럼별 최대 문자열 길이 검사

데이터와 함께 제공된 스키마와 비교하여 데이터에 존재하는 최대 길이를 넘지 않는지 검증

(데이터 로딩이 잘못된 경우 최대 길이를 넘는 컬럼이 존재할 수 있어 확인함)

(내부DB에서 표제부와 폐말소대장이 하나의 테이블로 합쳐진 것은 개방데이터와는 무관함 확인)

In [5]:
# Query to count the occurrences
query = """
    SELECT 
        COUNT(관리_건축물대장_PK) AS 관리_건축물대장_PK_count,
        COUNT(관리_상위_건축물대장_PK) AS 관리_상위_건축물대장_PK_count
    FROM 기본개요
"""

# Execute the query and fetch the result
result = con.execute(query).fetchdf()
print(result)

   관리_건축물대장_PK_count  관리_상위_건축물대장_PK_count
0           27994157              21493611


(기본개요에서 각 컬럼별 널이 아닌 값 수를 셈)

In [8]:
# 1) 테이블 스키마에서 컬럼 이름을 가져오고
cols_df = con.execute("PRAGMA table_info('기본개요')").fetchdf()
cols = [
    "관리_건축물대장_PK",
    "지역_코드",
    "지구_코드",
    "구역_코드",
]

# 2) 각 컬럼에 대한 COUNT 문을 생성
count_exprs = [f"COUNT({col}) AS {col}_count" for col in cols]

# 대장종류코드별로 널이 아닌 값 수를 셈
for code in [1, 2, 3]:
    print(f"\n대장종류코드 = {code} 인 경우")
    query = """
    SELECT
        {exprs}
    FROM 기본개요
    WHERE 대장_종류_코드 = {code}
    """.format(exprs=",\n    ".join(count_exprs), code=code)
    result = con.execute(query).fetchdf()
    print(result.T)


대장종류코드 = 1 인 경우
                        0
관리_건축물대장_PK_count  615572
지역_코드_count        405375
지구_코드_count         58031
구역_코드_count        107059

대장종류코드 = 2 인 경우
                         0
관리_건축물대장_PK_count  7383340
지역_코드_count        3708318
지구_코드_count         714918
구역_코드_count         806061

대장종류코드 = 3 인 경우
                        0
관리_건축물대장_PK_count  649017
지역_코드_count        311741
지구_코드_count         90388
구역_코드_count         56961


총괄표제부
관리_건축물대장_PK_count  615572
지역_코드_count        405375
지구_코드_count         58031
구역_코드_count        107059

일반건축물대장
관리_건축물대장_PK_count  7383340
지역_코드_count        3708318
지구_코드_count         714918
구역_코드_count         806061

표제부
관리_건축물대장_PK_count  649017
지역_코드_count        311741
지구_코드_count         90388
구역_코드_count         56961

In [ ]:
# 1) 테이블 스키마에서 컬럼 이름을 가져오고
cols_df = con.execute("PRAGMA table_info('표제부')").fetchdf()
cols = cols_df["name"].tolist()

# 2) 각 컬럼에 대한 COUNT 문을 생성
# 표제부에는 컬럼명에 %가 들어간 경우가 있어, 오류가 발생함
# 이를 피하기 위해, 컬럼명을 따옴표로 감싸줌
count_exprs = [f'COUNT("{col}") AS "{col}_count"' for col in cols]

# 3) 최종 쿼리 조립
query = """
SELECT
    {exprs}
FROM 표제부
""".format(exprs=",\n    ".join(count_exprs))

# 4) 실행 및 결과 확인
result = con.execute(query).fetchdf()

with pd.option_context(
    "display.max_rows",
    None,
    "display.max_columns",
    None,
):
    # Display the result with all rows and columns
    print(result.T)

                            0
관리_건축물대장_PK_count     8027067
대장_구분_코드_count        8027067
대장_구분_코드_명_count      8027067
대장_종류_코드_count        8027067
대장_종류_코드_명_count      8027067
대지_위치_count           8027067
도로명_대지_위치_count       7076491
건물_명_count            1160843
시군구_코드_count          8027067
법정동_코드_count          8027063
대지_구분_코드_count        8027059
번_count               8027067
지_count               8027055
특수지_명_count             33202
블록_count                12772
로트_count                 8734
외필지_수_count           8027067
새주소_도로_코드_count       7076491
새주소_법정동_코드_count      7074579
새주소_지상지하_코드_count     8024683
새주소_본_번_count         7174022
새주소_부_번_count         5375334
동_명_count             2199542
주_부속_구분_코드_count      8027067
주_부속_구분_코드_명_count    8026468
대지_면적(㎡)_count        8027067
건축_면적(㎡)_count        8027067
건폐_율(%)_count         8027067
연면적(㎡)_count          8027067
용적_률_산정_연면적(㎡)_count  8027067
용적_률(%)_count         8027067
구조_코드_count           8006871
구조_코드_명_co

(표제부에서 각 컬럼별 널이 아닌 값 수를 셈)
(표제부는 컬럼명에 특수문자(%)가 들어간 컬럼이 있어, 그대로 사용하면 에러가 발생함. 따옴표로 감싸줘야 함)